<a href="https://colab.research.google.com/github/sheldor07/Essay-Evaluator-Research-Assistantship/blob/main/topic_classification_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 69.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
from transformers import pipeline

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/essayData.csv')

# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


In [3]:
df.head()

,Essay,Section,ABSTRACT,terms
0,1,1,Four in five Singaporeans say that they can co...,humanities
1,1,1,While the terms ÃfalsehoodsÃ and Ãfake news...,humanities
2,1,2,The digital era is characterised by easy creat...,humanities
3,1,2,It is important to be critical of fake news du...,"sciences,humanities"
4,1,3,Fake news has played an enormous role during C...,humanities


In [12]:

# Create an empty list to store the predictions
predictions = []
word_labels = []
num_rows = df.shape[0]
# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Get the value from the "Section" column
    sequence_to_classify = row['ABSTRACT']
    # Define the candidate labels
    candidate_labels = ['technology', 'business', 'humanities', 'sciences']

    # Perform zero-shot classification and get the predicted label
    prediction = classifier(sequence_to_classify, candidate_labels)

    # Add the predicted label to the list
    word_labels.append(prediction['labels'][0])
    predictions.append(prediction['scores'])
    section = row['Section']
    print(f'row :{section}/{num_rows}')

# Add the predictions as a new column in the DataFrame
df['Predicted_Label'] = predictions
df['Max_Label'] = word_labels

# Save the updated DataFrame to a new CSV file
df.to_csv('predicted_file_topic_class.csv', index=False)


row :1/236
row :1/236
row :2/236
row :2/236
row :3/236
row :4/236
row :4/236
row :4/236
row :5/236
row :5/236
row :6/236
row :6/236
row :6/236
row :7/236
row :7/236
row :8/236
row :9/236
row :9/236
row :10/236
row :11/236
row :11/236
row :11/236
row :11/236
row :12/236
row :12/236
row :1/236
row :1/236
row :2/236
row :2/236
row :3/236
row :3/236
row :3/236
row :3/236
row :3/236
row :4/236
row :4/236
row :4/236
row :4/236
row :5/236
row :6/236
row :6/236
row :7/236
row :8/236
row :8/236
row :9/236
row :10/236
row :10/236
row :10/236
row :11/236
row :11/236
row :11/236
row :12/236
row :12/236
row :1/236
row :1/236
row :2/236
row :3/236
row :3/236
row :4/236
row :4/236
row :4/236
row :4/236
row :5/236
row :5/236
row :5/236
row :6/236
row :6/236
row :6/236
row :6/236
row :6/236
row :7/236
row :7/236
row :7/236
row :7/236
row :8/236
row :8/236
row :8/236
row :8/236
row :9/236
row :10/236
row :10/236
row :11/236
row :11/236
row :12/236
row :1/236
row :2/236
row :2/236
row :2/236
row :3/236
r

In [22]:

# Calculate the accuracy
correct_predictions = 0
for index,row in df.iterrows():
  max_label =row['Max_Label']
  terms = row['terms']
  if(max_label in terms):
    correct_predictions+=1
total_predictions = len(df)
accuracy = correct_predictions / total_predictions

# Print the accuracy
print("Accuracy:", accuracy)


Accuracy: 0.711864406779661
